<h2>Generate entireCustomers.csv and perfectCustomers.csv</h2>
<p>using month by month customer energy usage data contained in 'raw'.</p>
<p>Identify the customers with perfect time series for each month of 2013 and create the
'perfectCustomers' csv file containing 'id', 'acorn', 'acorn-grouped'. </p>
<p>Identify the any customer with a time series for any month of 2013 and create the
'entireCustomers' csv file containing 'id', 'acorn', 'acorn-grouped'. </p>

In [3]:
headers=['id', 'pricing', 'datetime',
         'kWh', 'acorn', 'acorn-grouped']

def process_file(fpath):
    
    # Given a filename in 'fpath', returns a tuple containing:
    # 0. A list of customer ids for which there is a full month's time series.
    # 1. A dataframe containing columns 'id', 'acorn', 'acorn-grouped' for any
    #    customer for which a time series exists.
    
    df = pd.read_csv(fpath, names=headers, low_memory=False)
    
    # Take the 'MAC' identifier off the id.

    df['id'] = df['id'].apply(lambda x: int(x[3:]))
    
    # Get rid of duplicates.
    
    xdf = df.groupby(['id', 'datetime']).first()
    df = xdf.reset_index(level=1)
    
    # Curtail the datetime entries to look like dates,
    # so readings from the same day can be grouped together using 'date.'
    
    df['datetime'] = df['datetime'].astype(str).str[0:10]
    
    df.reset_index(inplace=True)
    
    # Log how many readings exist for each customer id.
    
    from collections import Counter
    axa = Counter(df['id'])
    
    # What is the maximum number of readings indicated for any customer id
    # from this month.
    
    maxval = max(axa.values())
    print(maxval)
    
    fullList = [a for (a,b) in axa.items() if b==maxval]
    
    dfx = df[['id', 'acorn', 'acorn-grouped']].groupby('id').first()
    
    return fullList, dfx


In [4]:
import pandas as pd
import numpy as np
import os

raw_dir = os.path.join('..', '..', 'raw')

# Build customerList - a list of customer id's, with each id appearing
# once for every month in 2013 for which it has a complete time series representation.

# Build dfacorn - a dataframe containing columns 'id', 'acorn', 'acorn-grouped'
# with each customer 'id' appearing once for each month that it has any time series
# representation.

customerList = []
dfacorn = pd.DataFrame()

with os.scandir(raw_dir) as it:
    for entry in it:   
        if entry.name[0:4] == '2013':
            print(entry.name)
            cl, dfa = process_file(entry.path)
            customerList.extend(cl)
            dfacorn = dfacorn.append(dfa)

2013-01-power-survey-london.csv.bz2
1488
2013-02-power-survey-london.csv.bz2
1344
2013-03-power-survey-london.csv.bz2
1488
2013-04-power-survey-london.csv.bz2
1440
2013-05-power-survey-london.csv.bz2
1488
2013-06-power-survey-london.csv.bz2
1440
2013-07-power-survey-london.csv.bz2
1488
2013-08-power-survey-london.csv.bz2
1488
2013-09-power-survey-london.csv.bz2
1440
2013-10-power-survey-london.csv.bz2
1488
2013-11-power-survey-london.csv.bz2
1440
2013-12-power-survey-london.csv.bz2
1488


In [5]:
# Determine how many times each customer appears in customerList. Those
# that appear all 12 times will be designated 'perfect' customers.

from collections import Counter
xcount = Counter(customerList)

In [6]:
# Form a list of ids for customers who have a complete record over the 12 months of 2013.

customerList = [a for (a,b) in xcount.items() if b==12]

In [7]:
# Reminder - dfacorn is a dataframe containing columns 'id', 'acorn', 'acorn-grouped',
# for each customer id with a time series representation in 2013. There may be duplicate
# instances of customer id's. 

# Create the entire customers dataframe, making sure we only have one row for each customer..

entireCustomers = dfacorn.reset_index(level=0).groupby(['id']).first()

In [8]:
# Create the perfect customers dataframe.

actmp = entireCustomers.reset_index(level=0)
perfectCustomers = actmp.loc[actmp['id'].isin(customerList),:]

In [9]:
# Export the entire and perfect customer lists in csv format.

entireCustomers.to_csv('../../refined/entireCustomers.csv')
perfectCustomers.to_csv('../../refined/perfectCustomers.csv', index=False)